In [26]:
from sklearn.preprocessing import StandardScaler

In [3]:
# -*- coding:utf-8 -*-
import numpy as np
import sys
sys.path.append('../scripts')
from obtain_features import ObtainFeatures

def create_features(acc_raw, gyro_raw):    
    of = ObtainFeatures(acc_raw, gyro_raw)
    
    # Remove noises by median filter & Butterworth filter
    acc_raw = of.apply_median_filter(acc_raw)
    acc_raw = of.apply_butterworth_filter(acc_raw)
    gyro_raw = of.apply_median_filter(gyro_raw)
    gyro_raw = of.apply_butterworth_filter(gyro_raw)
    
    # Sample signals in fixed-width sliding windows
    tAccXYZ, tBodyGyroXYZ = of.segment_signal(acc_raw, gyro_raw)
    
    return tAccXYZ, tBodyGyroXYZ

In [41]:
# -*- coding:utf-8 -*-
import logging
from datetime import datetime
import numpy as np
import pandas as pd
import glob


# Create log file
EXEC_TIME = 'create-raw-data-for-dl-' + datetime.now().strftime("%Y%m%d-%H%M%S")
logging.basicConfig(filename=f'../logs/{EXEC_TIME}.log', level=logging.DEBUG)
logging.debug(f'../logs/{EXEC_TIME}.log')


def create_raw_data():
    logging.debug('Start creating raw datasets')
    
    train_subjects = [1, 3, 5, 6, 7, 8, 11, 14, 15, 16, 17, 19, 21, 22, 23, 25, 26, 27, 28, 29, 30]
    test_subjects = [2, 4, 9, 10, 12, 13, 18, 20, 24]
    
    root = '../data/hapt_data_set/RawData/'
    acc_files = sorted(glob.glob(root + 'acc*.txt'))
    gyro_files = sorted(glob.glob(root + 'gyro*.txt'))
    
    label_info = pd.read_table(root + 'labels.txt', sep=' ', header=None, names=['ExpID', 'UserID', 'ActID', 'ActStart', 'ActEnd'])
    
    X_train = np.array([])
    X_test = np.array([])
    
    for acc_file, gyro_file in zip(acc_files, gyro_files):
        exp_id = int(acc_file.split('exp')[1][:2])
        user_id = int(acc_file.split('user')[1][:2])
        
        print(f'User ID: {user_id}')
        
        temp_label_info = label_info[(label_info.ExpID==exp_id)&(label_info.UserID==user_id)&(label_info.ActID.isin([1,2,3,4,5,6]))]
    
        acc_raw = pd.read_table(acc_file, sep=' ', header=None, names=['x', 'y', 'z'])
        gyro_raw = pd.read_table(gyro_file, sep=' ', header=None, names=['x', 'y', 'z'])
        
        acc_raw = pd.DataFrame(StandardScaler().fit_transform(acc_raw), columns=['x', 'y', 'z'])
        gyro_raw = pd.DataFrame(StandardScaler().fit_transform(gyro_raw), columns=['x', 'y', 'z'])
        
        for _, _, act_id, act_start, act_end in temp_label_info.values:
            temp_acc_raw = acc_raw.iloc[act_start:act_end+1]
            temp_gyro_raw = gyro_raw.iloc[act_start:act_end+1]
            tAccXYZ, tBodyGyroXYZ = create_features(temp_acc_raw, temp_gyro_raw)
            features = np.zeros((len(tAccXYZ), 128, 6))
            
            for i in range(len(tAccXYZ)):
                feature = pd.DataFrame(np.concatenate((tAccXYZ[i], tBodyGyroXYZ[i]), 1), columns=['AccX', 'AccY', 'AccZ', 'GyroX', 'GyroY', 'GyroZ'])
                features[i] = feature
            
            if user_id in train_subjects:
                if len(X_train) == 0:
                    X_train = features
                else:
                    X_train = np.vstack((X_train, features))
            else:
                if len(X_test) == 0:
                    X_test = features
                else:
                    X_test = np.vstack((X_test, features))
    
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 6, 1)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 6, 1)
    print(f'X_train {X_train.shape}')
    print(f'X_test {X_test.shape}')
    
    np.save('../data/my_dataset/Raw_X_train', X_train)
    np.save('../data/my_dataset/Raw_X_test', X_test)

In [42]:
create_raw_data()

User ID: 1
User ID: 1
User ID: 2
User ID: 2
User ID: 3
User ID: 3
User ID: 4
User ID: 4
User ID: 5
User ID: 5
User ID: 6
User ID: 6
User ID: 7
User ID: 7
User ID: 8
User ID: 8
User ID: 9
User ID: 9
User ID: 10
User ID: 10
User ID: 10
User ID: 11
User ID: 11
User ID: 12
User ID: 12
User ID: 13
User ID: 13
User ID: 14
User ID: 14
User ID: 15
User ID: 15
User ID: 16
User ID: 16
User ID: 17
User ID: 17
User ID: 18
User ID: 18
User ID: 19
User ID: 19
User ID: 20
User ID: 20
User ID: 21
User ID: 21
User ID: 22
User ID: 22
User ID: 23
User ID: 23
User ID: 24
User ID: 24
User ID: 25
User ID: 25
User ID: 26
User ID: 26
User ID: 27
User ID: 27
User ID: 28
User ID: 28
User ID: 29
User ID: 29
User ID: 30
User ID: 30
X_train (7406, 128, 6, 1)
X_test (2993, 128, 6, 1)
